In [ ]:
# 2_modeling.py

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate
import joblib

# Load data
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv").squeeze()

# Models
models = {
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(random_state=42, max_iter=1000))
    ]),
    "Random Forest": RandomForestClassifier(
        n_estimators=100, max_depth=10, min_samples_split=10, random_state=42
    ),
    "Gradient Boosting": GradientBoostingClassifier(
        n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42
    )
}

# Cross-validation metrics
scoring = ["accuracy", "precision", "recall", "f1", "roc_auc"]
cv_results = {}

print("Evaluating models with 5-fold cross-validation...")
for name, model in models.items():
    scores = cross_validate(model, X_train, y_train, cv=5, scoring=scoring)
    cv_results[name] = scores
    print(f"\n{name}:")
    for metric in scoring:
        mean = scores[f"test_{metric}"].mean()
        std = scores[f"test_{metric}"].std()
        print(f"{metric.capitalize():10}: {mean:.4f} ± {std:.4f}")

# Fit and save models
for name, model in models.items():
    model.fit(X_train, y_train)
    joblib.dump(model, f"outputs/models/{name.replace(' ', '_').lower()}.pkl")

# Feature importance for Random Forest
rf_model = models["Random Forest"]
importances = pd.DataFrame({
    "feature": X_train.columns,
    "importance": rf_model.feature_importances_
}).sort_values("importance", ascending=False)

importances.to_csv("outputs/feature_importance_rf.csv", index=False)
print("\nModel training complete. Models and feature importance saved.")